# Prediction of Key Variables in Wastewater Treatment Plants Using Machine Learning Models

## Case Studies - Transformer Algorithm

## Notebook developed for WCCI paper case study: <p style="color:blue"> Biological Oxygen Demand (BOD5).</p>

## Data source: Benchmark Simulation Model No 2 - BSM2

## Objective: 
Predict BOD5 value at the entrance of aerobic tank number 3

As the objective is to predict the BOD value at the entrance of the aerobic tank, the data will be collected between units (tanks) 2 and 3.

## Initial Exploratory Data Analysis

The initial exploratory analysis was performed on the main notebook for the case study.

In [1]:
# Load required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, datetime
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
print('Tensorflow version: {}'.format(tf.__version__))
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore')

Tensorflow version: 2.6.0


In [3]:
# # Load datasets
df_entrada = pd.read_csv('datasets/dado3.csv')

In [4]:
# Delete 5 dummy variables
df_entrada.drop(["d1", "d2","d3","dp1","dp2"], axis = 1, inplace = True)

In [5]:
# Formula to approximate the value of BOD5
df_entrada['CBO5'] = 0.25*(df_entrada['S_S'] + df_entrada['X_S'] + (1-0.08)*(df_entrada['X_b,h'] + df_entrada['X_b,a']))

In [6]:
df1 = df_entrada.filter(['X_b,h', 'X_p', 'S_0','CBO5', 'S_ND'], axis=1)

df = df1[0:17280] # 180 days for training and testing
val_data = df1[34744:36001] # 10 days (holdout sample) for final test

## Algorithms: Transformers

In [7]:
batch_size = 32
seq_len = 96
d_k = 256
d_v = 256
n_heads = 12
ff_dim = 256

In [8]:
# Normalize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scalerT = scaler.fit(df)
train_data = scalerT.transform(df)

In [9]:
# Training data
X_train, y_train = [], []
for i in range(seq_len, len(train_data)):
    X_train.append(train_data[i-seq_len:i]) 
    y_train.append(train_data[:, 3][i])
X_train, y_train = np.array(X_train), np.array(y_train)

In [10]:
# Normalize Validation data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scalerV = scaler.fit(val_data)
val_data = scalerV.transform(val_data)

In [11]:
# Validation data
X_val, y_val = [], []
for i in range(seq_len, len(val_data)):
    X_val.append(val_data[i-seq_len:i])
    y_val.append(val_data[:, 3][i])
X_val, y_val = np.array(X_val), np.array(y_val)

In [12]:
print('Training set shape', X_train.shape, y_train.shape)
print('Validation set shape', X_val.shape, y_val.shape)

Training set shape (17184, 96, 5) (17184,)
Validation set shape (1161, 96, 5) (1161,)


In [13]:
class Time2Vector(Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        '''Initialize weights and biases with shape (batch, seq_len)'''
        self.weights_linear = self.add_weight(name='weight_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_linear = self.add_weight(name='bias_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.weights_periodic = self.add_weight(name='weight_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

    def call(self, x):
        '''Calculate linear and periodic time features'''
        x = tf.math.reduce_mean(x[:,:,:4], axis=-1) 
        time_linear = self.weights_linear * x + self.bias_linear # Linear time feature
        time_linear = tf.expand_dims(time_linear, axis=-1) # Add dimension (batch, seq_len, 1)

        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # Add dimension (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # shape = (batch, seq_len, 2)
   
    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'seq_len': self.seq_len})
        return config

In [14]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                           input_shape=input_shape, 
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

        self.key = Dense(self.d_k, 
                         input_shape=input_shape, 
                         kernel_initializer='glorot_uniform', 
                         bias_initializer='glorot_uniform')

        self.value = Dense(self.d_v, 
                           input_shape=input_shape, 
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)

        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    

#############################################################################

class MultiAttention(Layer):
    def __init__(self, d_k, d_v, n_heads):
        super(MultiAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.attn_heads = list()

    def build(self, input_shape):
        for n in range(self.n_heads):
            self.attn_heads.append(SingleAttention(self.d_k, self.d_v))  

        # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1]=7 
        self.linear = Dense(input_shape[0][-1], 
                            input_shape=input_shape, 
                            kernel_initializer='glorot_uniform', 
                            bias_initializer='glorot_uniform')

    def call(self, inputs):
        attn = [self.attn_heads[i](inputs) for i in range(self.n_heads)]
        concat_attn = tf.concat(attn, axis=-1)
        multi_linear = self.linear(concat_attn)
        return multi_linear   

#############################################################################

class TransformerEncoder(Layer):
    def __init__(self, d_k, d_v, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.ff_dim = ff_dim
        self.attn_heads = list()
        self.dropout_rate = dropout

    def build(self, input_shape):
        self.attn_multi = MultiAttention(self.d_k, self.d_v, self.n_heads)
        self.attn_dropout = Dropout(self.dropout_rate)
        self.attn_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)

        self.ff_conv1D_1 = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')
        # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1] = 7 
        self.ff_conv1D_2 = Conv1D(filters=input_shape[0][-1], kernel_size=1) 
        self.ff_dropout = Dropout(self.dropout_rate)
        self.ff_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)    
  
    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        attn_layer = self.attn_multi(inputs)
        attn_layer = self.attn_dropout(attn_layer)
        attn_layer = self.attn_normalize(inputs[0] + attn_layer)

        ff_layer = self.ff_conv1D_1(attn_layer)
        ff_layer = self.ff_conv1D_2(ff_layer)
        ff_layer = self.ff_dropout(ff_layer)
        ff_layer = self.ff_normalize(inputs[0] + ff_layer)
        return ff_layer 

    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                       'd_v': self.d_v,
                       'n_heads': self.n_heads,
                       'ff_dim': self.ff_dim,
                       'attn_heads': self.attn_heads,
                       'dropout_rate': self.dropout_rate})
        return config          

In [17]:
# If the algorithm has already been created and trained, you don't need to run this part.

def create_model():
    # Initialize time and transformer layers
    time_embedding = Time2Vector(seq_len)
    attn_layer1 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
    attn_layer2 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
    attn_layer3 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)

    # Construct model
    in_seq = Input(shape=(seq_len, 5))
    x = time_embedding(in_seq)
    x = Concatenate(axis=-1)([in_seq, x])
    x = attn_layer1((x, x, x))
    x = attn_layer2((x, x, x))
    x = attn_layer3((x, x, x))
    x = GlobalAveragePooling1D(data_format='channels_first')(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    out = Dense(1, activation='linear')(x)

    model = Model(inputs=in_seq, outputs=out)
    model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mape'])
    return model


model = create_model()
model.summary()

callback = tf.keras.callbacks.ModelCheckpoint('Transformer_BOD5.hdf5', 
                                              monitor='val_loss', 
                                              save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=40, 
                    callbacks=[callback],
                    validation_data=(X_val, y_val))  

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 5)]      0                                            
__________________________________________________________________________________________________
time2_vector_1 (Time2Vector)    (None, 96, 2)        384         input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 96, 7)        0           input_2[0][0]                    
                                                                 time2_vector_1[0][0]             
__________________________________________________________________________________________________
transformer_encoder_3 (Transfor (None, 96, 7)        99114       concatenate_1[0][0]        

Epoch 21/40
537/537 [==============================] - 609s 1s/step - loss: 0.0121 - mae: 0.0830 - mape: 42.9134 - val_loss: 0.0212 - val_mae: 0.1143 - val_mape: 45.9497

Epoch 00021: val_loss did not improve from 0.02042
Epoch 22/40
537/537 [==============================] - 608s 1s/step - loss: 0.0113 - mae: 0.0801 - mape: 41.9240 - val_loss: 0.0245 - val_mae: 0.1134 - val_mape: 50.5127

Epoch 00022: val_loss did not improve from 0.02042
Epoch 23/40
537/537 [==============================] - 608s 1s/step - loss: 0.0112 - mae: 0.0798 - mape: 43.0627 - val_loss: 0.0193 - val_mae: 0.1063 - val_mape: 52.4097

Epoch 00023: val_loss improved from 0.02042 to 0.01930, saving model to Transformer_BOD5.hdf5
Epoch 24/40
537/537 [==============================] - 604s 1s/step - loss: 0.0115 - mae: 0.0802 - mape: 42.6217 - val_loss: 0.0284 - val_mae: 0.1195 - val_mape: 58.9134

Epoch 00024: val_loss did not improve from 0.01930
Epoch 25/40
537/537 [==============================] - 607s 1s/step -

In [18]:
# Load the trained model
model = tf.keras.models.load_model('modelos/Transformer_BOD5.hdf5',
                                   custom_objects={'Time2Vector': Time2Vector, 
                                                   'SingleAttention': SingleAttention,
                                                   'MultiAttention': MultiAttention,
                                                   'TransformerEncoder': TransformerEncoder})

In [ ]:
'''Calculate predictions and metrics'''

#Calculate predication for training, validation and test data
train_pred = model.predict(X_val)

#X_val, y_val
#Print evaluation metrics for all datasets
train_eval = model.evaluate(X_val, y_val, verbose=0)

print(' ')
print('Evaluation metrics')
print('Training Data - Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}'.format(train_eval[0], train_eval[1], train_eval[2]))


###############################################################################
'''Display results'''

fig = plt.figure(figsize=(15,20))
st = fig.suptitle("Transformer", fontsize=22)
st.set_y(0.92)

#Plot training data results
ax11 = fig.add_subplot(311)
ax11.plot(val_data[:, 3], label='BOD5_real')
ax11.plot(np.arange(seq_len, train_pred.shape[0]+seq_len), train_pred, linewidth=1, label='BOD5_pred')
ax11.set_title("BOD5_target", fontsize=18)
ax11.set_xlabel('BOD5_target')
ax11.set_ylabel('BOD5_target')
ax11.legend(loc="best", fontsize=12)

In [21]:
# Make prediction with trained model
previsao_trans = model.predict(X_val)

In [22]:
# Copy scaler dimension to do inverse normalization
prediction_copies = np.repeat(previsao_trans , val_data.shape[1], axis=-1)
previstoT = scalerV.inverse_transform(prediction_copies)[:,3] # valor previsto

In [23]:
previstoT.shape

(1161,)

In [24]:
# Load data with original format
df1 = df_entrada.filter(['X_b,h', 'X_p', 'S_0','CBO5', 'S_ND'], axis=1)


df = df1[0:17280] 
val_data = df1[34744:36001] 

In [25]:
# Create dataframe with actual and forecast values
df2 = val_data.CBO5
df2 = df2.reset_index(drop=True)

In [26]:
df3 = df2[96:1257]
df3 = df3.reset_index(drop=True)

In [27]:
df3.shape

(1161,)

In [32]:
df4 = df3[96:1161]
df4 = df4.reset_index(drop=True)

In [37]:
prevT = previstoT[96:1161]

In [2]:
# BOD5 graph
plt.figure(figsize = (15, 6))
plt.plot(df4) 
plt.plot(prevT) # em verde 
plt.title('Transformer', family='Arial', fontsize=14)
plt.xlabel('Samples(15 minutos)')
plt.ylabel('BOD5 value')
plt.legend(['Ral value', 'Predicted value'], loc='upper right')

In [39]:
# Export forecast values
pd.DataFrame(prevT).to_csv('previsao_CBO5.csv')